### **Sentinel 2 cloud masking with s2cloudless on Utah lake**

By using S2 cloudless algorithm you can calculate per-pixel cloud probability for Sentinel 2 image at 10 m scale. Following steps are used to remove clouds and shadows from an image. 
--First import and filter out surface reflectance (SR) and S2 cloudless collection. Join the filtered s2cloudless collection to the SR collection.
--Then add S2 cloudless probability layer and derive cloud masks to an input S2 SR input image. Subset the probability band and use probability threshold value. Add cloud probability layer and cloud mask as image bands
--Add cloud projection and dark pixels by creating Scene classification Layer (SCL) which distinguishes between cloudy, clear and water pixels. SCL bit value of water is 6 which identifies water pixel from SCL band. Identify dark NIR pixels that are not water. Determine the direction to project cloud shadow from clouds. Identify the intersection of dark pixels with cloud shadow projection. Add dark pixels, cloud projection, and identified shadows as image bands. 
--Add cloud and cloud shadow component bands Assemble all cloud and cloud shadow components and create final layer. 
--Define a method for displaying Earth Engine image tiles to a folium map. Folium is used to display map layers. Add EE layer method to folium. 
--Display all cloud and shadow components to an interactive folium map. Once the folium map is generated, you can toggle layers on and off.
-Try changing cloud probability threshold, NIR dark threshold, cloud projection distance and buffer input variables by rerunning the algorithm and see how the result changes. 


The s2cloudless image provides a cloud presence probability between 0 and 100 percent that can be used to customize the aggressiveness of cloud masking procedure. There is always balancing act between commission and omission errors, but you can optimize masking according to needs for the project. You can also select probability threshold for defining cloud and non cloud masks. 

In [76]:
#! pip install geemap

In [30]:
# Importing libraries
import os
import ee
import geemap

In [75]:
# Initializing EarthEngine API
#ee.Authenticate()

# Initialize the library.
#ee.Initialize()

In [34]:
Map = geemap.Map()

aoi = ee.Geometry.Polygon([
[-111.9087690270072,40.34318724860489],
[-111.91151560903845,40.3253910740408],
[-111.88267649771032,40.29921170364489],
[-111.85658396841345,40.26254353683722],
[-111.86345042349157,40.2279524800051],
[-111.8867963707572,40.198588631809],
[-111.91700877310095,40.162915427153834],
[-111.9417280113822,40.134573345328256],
[-111.93760813833532,40.116722264048725],
[-111.92112864614782,40.09991696446809],
[-111.92250193716345,40.08310751316512],
[-111.90464915396032,40.06419191836848],
[-111.8922895348197,40.041065723892224],
[-111.8813032066947,40.03686011821013],
[-111.86345042349157,40.05998773941388],
[-111.8648237145072,40.07154860728996],
[-111.85109080435095,40.09991696446809],
[-111.80577220083532,40.14297223209199],
[-111.80027903677282,40.13352341148702],
[-111.74122752310095,40.158717346462666],
[-111.72337473989782,40.17550811171782],
[-111.72612132192907,40.20488195606535],
[-111.73436106802282,40.23529145396199],
[-111.73024119497595,40.258351622924415],
[-111.75770701528845,40.277213190093825],
[-111.76320017935095,40.32120305678459],
[-111.76457347036657,40.33481316286385],
[-111.78654612661657,40.34004708825586],
[-111.83598460317907,40.357839397527975],
[-111.88954295278845,40.358885857780336],
[-111.9087690270072,40.34318724860489]                                
])


start_date = '2021-06-01'
end_date = '2021-06-02'
cloud_filter = 60 # max cloud cover % allowed in image
cld_prb_thresh = 50 # greater than 50% considered as clouds
nir_drk_thresh = 0.10 # values less than 0.15 are considered as clouds
cld_prj_dist = 2 # max distance from cloud edges
buffer = 70
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [32]:
# Define function to filter SR and S2 cloudless collections

def get_S2_SR_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    S2_SR_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))

    # Import and filter s2cloudless.
    S2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': S2_SR_col,
        'secondary': S2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [36]:
# Applying the function 
S2_SR_cld_col_eval = get_S2_SR_cld_col(aoi, start_date, end_date)

In [37]:
#Defining a function to add S2 cluodless probability layer and derive cloud masks to S2SR input image

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(cld_prb_thresh).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [50]:
#Defining a function to add cloud projection and dark pixels

def add_shadow_bands(img):
     
    # Scene Classification Layer (SCL) distinguishes between cloudy, clear and water pixels. 
    # Identify water pixels from the SCL band. 
    # Note that SCL bit value 6 is water


    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(nir_drk_thresh*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    # Note that the peak solar irradiance is when the sun is overhead. It happens around noon (11:00 PM to 2:00 PM), and the solar elevation angle reaches 90°. 
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the cloud project distance input.
    # Note that for each zero-values pixel in the source directionalDistanceTransform method get the distance to the nearest non-zero pixel in the given direction.
    # It returns the band of floating point distance called 'distance',
    
    
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, cld_prj_dist*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [51]:
# Defining a function to assemble all of the cloud and cloud shadow components and create the final mask.

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(buffer*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20}) # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [52]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [53]:
# Defining a function to display all of the cloud and shadow components in an interactive folium map

def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = aoi.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

In [1]:
# Display mask component layers
#S2_SR_cld_col_eval_disp = S2_SR_cld_col_eval.map(add_cld_shdw_mask)

#display_cloud_layers(S2_SR_cld_col_eval_disp)

In [70]:
Map = geemap.Map()

AOI = ee.Geometry.Polygon([
[-111.9087690270072,40.34318724860489],
[-111.91151560903845,40.3253910740408],
[-111.88267649771032,40.29921170364489],
[-111.85658396841345,40.26254353683722],
[-111.86345042349157,40.2279524800051],
[-111.8867963707572,40.198588631809],
[-111.91700877310095,40.162915427153834],
[-111.9417280113822,40.134573345328256],
[-111.93760813833532,40.116722264048725],
[-111.92112864614782,40.09991696446809],
[-111.92250193716345,40.08310751316512],
[-111.90464915396032,40.06419191836848],
[-111.8922895348197,40.041065723892224],
[-111.8813032066947,40.03686011821013],
[-111.86345042349157,40.05998773941388],
[-111.8648237145072,40.07154860728996],
[-111.85109080435095,40.09991696446809],
[-111.80577220083532,40.14297223209199],
[-111.80027903677282,40.13352341148702],
[-111.74122752310095,40.158717346462666],
[-111.72337473989782,40.17550811171782],
[-111.72612132192907,40.20488195606535],
[-111.73436106802282,40.23529145396199],
[-111.73024119497595,40.258351622924415],
[-111.75770701528845,40.277213190093825],
[-111.76320017935095,40.32120305678459],
[-111.76457347036657,40.33481316286385],
[-111.78654612661657,40.34004708825586],
[-111.83598460317907,40.357839397527975],
[-111.88954295278845,40.358885857780336],
[-111.9087690270072,40.34318724860489]                                
])



start_date = '2021-03-01'
end_date = '2021-09-02'
cloud_filter = 60 # max cloud cover % allowed in image
cld_prb_thresh = 40 # greater than 40% considered as clouds
nir_drk_thresh = 0.15 # values less than 0.15 are considered as clouds
cld_prj_dist = 2 # max distance from cloud edges
buffer = 100


In [71]:
# Build S2 cloudless collection

S2_SR_cld_col = get_S2_SR_cld_col(aoi, start_date, end_date)

In [72]:
# Applying cloud and mask to each image

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [73]:
# Add cloud and cloud shadow component bands to each image

S2_SR_median = (S2_SR_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask) # Apply mask to each collection
                             .median()) # Reduce the collection by median

In [74]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(S2_SR_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)